In [ ]:
#default_exp parsing

# Parsing votes
> Downloading & parsing votes Aafter downloading xlsx files behind the links on `https://www.bundestag.de/parlament/plenum/abstimmung/liste`.

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup

In [ ]:
#export
from bs4 import BeautifulSoup
from pathlib import Path
import re, os
import typing
import requests
import tqdm
import time
import pandas as pd
import itertools

## Collecting URIs for `xlsx` documents from `htm` files

In [ ]:
html_data_dir = Path('../raw_data')
xlsx_data_dir = Path('../xlsx_data')

In [ ]:
#export
def get_file_paths(data_dir:typing.Union[Path,str], suffix:str=None, pattern=None): 
    return list(set([data_dir/f for f in os.listdir(data_dir) if (suffix and f.endswith(suffix)) or (pattern and pattern.search(f))]))

In [ ]:
html_file_paths = get_file_paths(html_data_dir, suffix='.htm')
html_file_paths[:3]

In [ ]:
html_file_paths = get_file_paths(html_data_dir, pattern=re.compile('(\.htm)'))
html_file_paths[:3]

In [ ]:
#export
def collect_xlsx_uris(html_file_paths:typing.List[Path]):
    xlsx_uris = {}
    pattern = re.compile('(XLSX?)')

    for file_path in html_file_paths:
        
        with open(file_path, 'r') as f:
            soup = BeautifulSoup(f)
        
        elements = soup.find_all('td', attrs={'data-th':'Dokument'})
        for element in elements:
            title = element.div.p.strong.text.strip()
            href = element.find('a', attrs={'title':pattern})
            if href is None: continue
            xlsx_uris[title] = href['href']
    return xlsx_uris

In [ ]:
%%time
xlsx_uris = collect_xlsx_uris(html_file_paths)
list(xlsx_uris.items())[:3], list(xlsx_uris.items())[-3:]

## Download `xlsx` files

In [ ]:
#export
def download_xlsx_file(xlsx_uri:str, xlsx_dir:typing.Union[Path,str]=Path('../xlsx_data'),
                  verbose:bool=False):
    fname = xlsx_uri.split('/')[-1]
    xlsx_dir = Path(xlsx_dir)
    with open(xlsx_dir/fname, 'wb') as f:
        r = requests.get(xlsx_uri)
        if verbose: print(f'Writing to {xlsx_dir/fname}')
        f.write(r.content)

In [ ]:
xlsx_uris.keys()

In [ ]:
xlsx_uris.values()

In [ ]:
%%time
xlsx_uri = xlsx_uris['10.09.2020: Abstrakte Normenkontrolle - Düngeverordnung (Beschlussempfehlung)']
download_xlsx_file(xlsx_uri, xlsx_dir=xlsx_data_dir, verbose=True)

In [ ]:
#export
def download_multiple_xlsx_files(xlsx_uris:typing.Dict[str,str], xlsx_dir:typing.Union[Path,str]=Path('../xlsx_data'),
                            t_sleep:float=.01):
    xlsx_file_title_maps = {}
    for xlsx_title, xlsx_uri in tqdm.tqdm(xlsx_uris.items(), desc='xlsx File', total=len(xlsx_uris)):
        download_xlsx_file(xlsx_uri, xlsx_dir=xlsx_dir)
        xlsx_file_title_maps[xlsx_uri.split('/')[-1]] = xlsx_title
        time.sleep(t_sleep)
    return xlsx_file_title_maps

In [ ]:
%%time
xlsx_file_title_maps = download_multiple_xlsx_files(xlsx_uris, xlsx_dir=xlsx_data_dir)

In [ ]:
xlsx_file_title_maps

## Load xlsx into DataFrames

In [ ]:
xlsx_files = get_file_paths(xlsx_data_dir, pattern=re.compile('(\.xlsx?)'))
xlsx_files[:3], xlsx_files[-3:]

In [ ]:
#export
def is_date(s:str, fun:typing.Callable):
    try:
        _ = fun(s)
        return True
    except:
        return False

def get_xlsx_df(xlsx_file:typing.Union[str,Path], xlsx_file_title_maps:typing.Dict[str,str]=None): 
    dfs = pd.read_excel(xlsx_file, sheet_name=None)
    assert len(dfs) == 1, 'The xlsx file has more than one page, that\' unexpected.'
    for name, df in dfs.items():
        df['sheet_name'] = name
        #df['date'] = pd.to_datetime(xlsx_file.name.split('_')[0])
    if xlsx_file_title_maps is not None:
        tmp = xlsx_file_title_maps[xlsx_file.name].split(':')
        date = tmp[0]
        if is_date(date, lambda x: pd.to_datetime(x, dayfirst=True)):
            date = pd.to_datetime(date, dayfirst=True)
            title = ':'.join(tmp[1:])
        elif is_date(xlsx_file.name.split('_')[0], pd.to_datetime):
            date = pd.to_datetime(xlsx_file.name.split('_')[0])
            title = xlsx_file_title_maps[xlsx_file.name]
        else:
            date = None
            title = xlsx_file_title_maps[xlsx_file.name]
            
        df['date'] = date
        df['title'] = title
    return df

In [ ]:
%%time
xlsx_file = xlsx_files[0]
get_xlsx_df(xlsx_file, xlsx_file_title_maps=xlsx_file_title_maps)

In [ ]:
#hide
assert isinstance(get_xlsx_df(xlsx_file), pd.DataFrame)

In [ ]:
#export
def collect_xlsxs_as_dict(xlsx_files:typing.List[typing.Union[Path,str]],
                          xlsx_file_title_maps:typing.Dict[str,str]=None): 
    df = []
    for xlsx_file in xlsx_files:
        df.append(get_xlsx_df(xlsx_file, xlsx_file_title_maps=xlsx_file_title_maps))
    return pd.concat(df, ignore_index=True)

In [ ]:
%%time
df = collect_xlsxs_as_dict(xlsx_files, xlsx_file_title_maps=xlsx_file_title_maps)

In [ ]:
%%time
#df.to_parquet('../votes.parquet')